In [1]:
#!pip install langchain
!pip install --upgrade --quiet  langchain-openai langchain_community tiktoken chromadb langchain
!pip install --upgrade --quiet  youtube-transcript-api
!pip install pytube




ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.16.4 requires beautifulsoup4, which is not installed.
nbconvert 7.16.4 requires bleach!=5.0.0, which is not installed.
nbconvert 7.16.4 requires defusedxml, which is not installed.
nbconvert 7.16.4 requires jupyterlab-pygments, which is not installed.
nbconvert 7.16.4 requires nbclient>=0.5.0, which is not installed.
nbconvert 7.16.4 requires pandocfilters>=1.4.1, which is not installed.
tensorboard 2.10.1 requires absl-py>=0.4, which is not installed.
tensorboard 2.10.1 requires google-auth-oauthlib<0.5,>=0.4.1, which is not installed.
tensorboard 2.10.1 requires markdown>=2.6.8, which is not installed.
tensorboard 2.10.1 requires tensorboard-data-server<0.7.0,>=0.6.0, which is not installed.
tensorboard 2.10.1 requires tensorboard-plugin-wit>=1.6.0, which is not installed.
tensorboard 2.10.1 requires

   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
   -------------- ------------------------- 20.5/57.6 kB 217.9 kB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.6 kB 372.4 kB/s eta 0:00:01
   ---------------------------------------- 57.6/57.6 kB 378.8 kB/s eta 0:00:00


In [20]:
import os
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

from dotenv import load_dotenv



In [21]:
load_dotenv()


True

# Transcript Examples

In [22]:
text = """[00:00.000 --> 00:03.900]  2024 is when I started to see a couple of examples of startups,
[00:03.900 --> 00:08.680]  especially from the recent YC badge, starting to pivot into web scraping.
[00:08.680 --> 00:11.060]  And I'm just here to try to connect the dots here as well.
[00:11.060 --> 00:13.140]  This probably has something to do with perplexity and
[00:13.140 --> 00:16.220]  the amount of interest in terms of we want to scrape the web so
[00:16.220 --> 00:21.220]  we can have the best, I guess, up-to-date answer for your LMs or
[00:21.220 --> 00:24.980]  best up-to-date search for a platform.
[00:26.260 --> 00:27.580]  Mendable is an example.
[00:27.600 --> 00:31.380]  So early days, if you ever were on Lang chain or
[00:31.380 --> 00:37.120]  Lama index documentation site, you might see a little robot icon in the corner.
[00:37.120 --> 00:40.760]  You click on it, you can do a natural language search query off of
[00:40.760 --> 00:41.800]  their documentation site.
[00:42.960 --> 00:45.280]  But they came out with this thing recently called Firecrawl,
[00:45.280 --> 00:50.560]  which is specifically for scraping the web using large language models.
[00:50.560 --> 00:53.580]  And we'll see a live example in a quick second.
[00:53.580 --> 00:56.840]  Gina AI, really cool company.
[00:56.840 --> 01:00.540]  Their policy is actually, they have embedding models, and
[01:00.540 --> 01:02.300]  I think no language models.
[01:02.300 --> 01:05.660]  But their embedding models, you can try them without having an API key.
[01:05.660 --> 01:07.940]  And I don't know who's footing the bill for this, but
[01:07.940 --> 01:11.820]  they keep coming out with really cool, free tools that you can try.
[01:11.820 --> 01:17.900]  One of them is reader API, which is, all you gotta do is append
[01:17.900 --> 01:22.940]  gnetai.com before any URL, and you're gonna get back some clean data from
[01:22.940 --> 01:24.260]  that website.
[01:24.260 --> 01:25.300]  It's almost mind-blowing.
[01:25.300 --> 01:32.320]  And last, I'm gonna show you this open source project called Scrape Graph AI.
[01:32.320 --> 01:36.800]  So this is a very elaborate orchestration of different Python
[01:36.800 --> 01:38.480]  modules that create graphs.
[01:38.480 --> 01:43.200]  So you can create a pipeline to scrape the web using large language models.
[01:43.200 --> 01:48.560]  So these two, it only give you back clean inputs, but
[01:48.560 --> 01:53.400]  this one actually incorporate AI in and answer your question at the very end.
[01:53.440 --> 01:57.000]  Or you can have ten different steps of what to do when you go to a website and
[01:57.000 --> 01:57.880]  scrape it.
[01:57.880 --> 02:01.160]  So what I'm gonna do today is I'm actually gonna be scraping my competitor's
[02:01.160 --> 02:02.560]  pricing pages.
[02:02.560 --> 02:05.720]  I'm doing this for myself, I'm building a product right now.
[02:05.720 --> 02:08.040]  And this matters to me.
[02:08.040 --> 02:10.560]  This is not a made up use case.
[02:10.560 --> 02:14.360]  I'm actually looking forward to see what's gonna come out of this.
[02:14.360 --> 02:18.640]  So I'm building in the learning and development space.
[02:18.680 --> 02:24.440]  So my competitors are obviously popular tools like Articulate 360,
[02:24.440 --> 02:30.040]  some challengers that are new to the market like 7Taps, Mindsmith,
[02:30.040 --> 02:30.960]  some other companies.
[02:30.960 --> 02:34.080]  But I have about four websites here today, and
[02:34.080 --> 02:36.880]  I'm keeping them all in a nice array.
[02:37.920 --> 02:41.880]  So what I'm gonna do is I'm just gonna run this to save it in memory.
[02:43.560 --> 02:47.120]  And the cool thing about this is that once I give you this, you can go home and
[02:47.120 --> 02:50.640]  play around and try to do some market research yourself.
[02:50.640 --> 02:58.040]  And now I'm also gonna set up this thing called TickToken.
[02:58.040 --> 02:59.800]  Do you guys know what TickToken is?
[02:59.800 --> 03:02.480]  I know you know what TickToken is, but I wanna let everyone else know what
[03:02.480 --> 03:03.120]  TickToken is.
[03:05.280 --> 03:09.800]  Okay, so for a large language model when it's being encoded,
[03:09.800 --> 03:12.920]  maybe you can explain this better than me, cuz I'm a software engineer.
[03:12.920 --> 03:15.280]  So and it depends on the model too, right?
[03:15.360 --> 03:19.600]  Dependent on what kind of tokenization or encoding mechanism that you use.
[03:19.600 --> 03:26.400]  For example, GPT-3 had a different encoding scheme than GPT-4 or GPT-4-0.
[03:26.400 --> 03:29.240]  That's why they were able to reduce the cost for
[03:29.240 --> 03:32.400]  newer generations a little bit just because of the way that tokenization works.
[03:33.440 --> 03:35.960]  And you get billed by the number of tokens.
[03:35.960 --> 03:37.280]  So less tokens, but
[03:37.280 --> 03:41.400]  the same sentence is usually cheaper for you as a consumer.
[03:41.400 --> 03:44.560]  So we're using TickToken here, which is straight from OpenAI.
[03:44.560 --> 03:49.960]  This is the exact library that OpenAI uses to encode their GPT models.
[03:49.960 --> 03:53.680]  We're using this not to create a new model, but we're using this to
[03:53.680 --> 03:58.680]  count the number of tokens that we're getting based on the scraped content
[03:58.680 --> 04:00.080]  of this website.
[04:00.080 --> 04:04.560]  So I wanna know how expensive it is for me to scrape all these things.
[04:04.560 --> 04:06.160]  I wanna convert that to a dollar amount.
[04:07.520 --> 04:11.880]  Comparing Beautiful Soup and Gina AI and Mendebol.
[04:11.880 --> 04:14.680]  See which one will save me the most amount of money.
[04:14.680 --> 04:18.880]  So I ran this and this is just an example sentence.
[04:18.880 --> 04:22.000]  What's the difference between a Beer Nuts and Deer Nuts?
[04:22.000 --> 04:26.400]  Beer Nuts are about $5, Deer Nuts are just under a buck.
[04:26.400 --> 04:27.760]  So, anybody get a joke?
[04:29.720 --> 04:35.520]  So this one cost about 0.000135 if you're using GPT-4-0.
[04:35.520 --> 04:39.320]  And the funny thing about OpenAI is that they didn't update TickToken for
[04:39.320 --> 04:40.800]  GPT-4-0 yet.
[04:40.800 --> 04:42.360]  So this is just a guesstimate.
[04:42.360 --> 04:46.840]  So last thing that I wanna set up is a pretty table.
[04:46.840 --> 04:49.200]  So pretty table is just library in Python.
[04:49.200 --> 04:51.960]  So you can make tables in your terminal.
[04:51.960 --> 04:57.240]  I want rows and columns so I can see which one is cost more than which.
[04:58.440 --> 05:01.160]  So that's all I'm doing here, very long function.
[05:01.160 --> 05:04.920]  All it's doing here is take into account scraped content,
[05:04.920 --> 05:07.760]  put them all in the right columns and rows.
[05:07.760 --> 05:08.920]  You can read this at home.
[05:09.680 --> 05:13.040]  But yeah, all right, so now let's set up the scrapers.
[05:13.040 --> 05:18.120]  First we're gonna install a good old friend, BeautifulSoup4.
[05:20.360 --> 05:23.920]  This is the most straightforward way to scrape any website.
[05:23.920 --> 05:28.400]  And probably the easiest way for them to detect that you're scraping and
[05:28.400 --> 05:29.440]  ban you.
[05:29.440 --> 05:34.080]  So it's not very sophisticated unless you bundle on a bunch of other tools.
[05:34.080 --> 05:38.360]  Okay, so we got a function to scrape the web with BeautifulSoup.
[05:38.400 --> 05:39.960]  We're not gonna use that yet.
[05:39.960 --> 05:42.520]  What we're gonna do is we're gonna run all of them at the same time.
[05:42.520 --> 05:44.360]  So I'm just gonna set up a bunch of stuff here.
[05:44.360 --> 05:46.240]  So here's Gina AI.
[05:46.240 --> 05:48.760]  This is what I'm talking about when I say this one is dead simple.
[05:50.480 --> 05:54.200]  All you gotta do is just add the string before the actual URL that you wanna
[05:54.200 --> 05:55.480]  scrape.
[05:55.480 --> 05:56.880]  And it's completely free.
[05:56.880 --> 05:59.800]  I don't know what the deal is, but use it, it's free.
[06:01.080 --> 06:02.120]  No strings attached.
[06:02.120 --> 06:05.960]  I mean, maybe they're trying to do some market research or something.
[06:05.960 --> 06:06.880]  But yeah, it's great.
[06:06.880 --> 06:08.880]  The jobs of VC money they could-
[06:08.880 --> 06:09.480]  Yeah.
[06:09.480 --> 06:10.000]  They could blow.
[06:11.400 --> 06:12.000]  Absolutely.
[06:13.400 --> 06:14.400]  Like free rides and Uber.
[06:15.720 --> 06:19.960]  In the early days, you just get free rides over and over.
[06:19.960 --> 06:22.560]  All right, last provider is Mendable.
[06:22.560 --> 06:24.240]  So Mendable, like I said recently,
[06:24.240 --> 06:27.880]  pivoted from, I guess not fully pivoted from, but
[06:27.880 --> 06:30.760]  they were doing documentation chatbots.
[06:30.760 --> 06:34.120]  So you go on like Langchain, Lama Index, all these companies.
[06:34.120 --> 06:36.520]  You can chat with a little chatbot in the bottom.
[06:36.520 --> 06:38.080]  So we're gonna try this one too.
[06:39.160 --> 06:40.680]  But this one requires an API key.
[06:42.760 --> 06:48.440]  And they gave everybody like, I think, 300 tokens for free.
[06:48.440 --> 06:52.240]  I'm not sure how many websites you can scrape with that.
[06:53.760 --> 06:55.520]  But-
[06:55.520 --> 06:56.560]  So from these other tools,
[06:56.560 --> 07:01.040]  the output is all text normalized or is it still pretty wrong?
[07:01.040 --> 07:03.120]  It's only in markdown.
[07:03.120 --> 07:07.880]  So they're all in markdown for some reason.
[07:07.880 --> 07:10.280]  Maybe because of large language models.
[07:10.280 --> 07:14.640]  Like they made these tools specifically to output markdown from
[07:14.640 --> 07:16.120]  a bunch of HTML tags.
[07:18.280 --> 07:18.780]  Yeah.
[07:21.200 --> 07:22.720]  So it's also like string.
[07:22.720 --> 07:24.760]  So there's no like, just pure string.
[07:24.760 --> 07:26.040]  You get a string back, like very long.
[07:28.440 --> 07:31.160]  Okay, this is the moment that we run everything.
[07:33.640 --> 07:34.600]  So I'm just gonna run this.
[07:36.040 --> 07:39.440]  All I'm doing here is just go on these websites,
[07:39.440 --> 07:42.720]  try to scrape it with all the three different tools that I have.
[07:43.720 --> 07:45.080]  It's gonna take about a minute.
[07:45.080 --> 07:49.760]  So we can watch this bar going from left to right.
[07:49.760 --> 07:50.880]  Are you guys gonna ask questions?
[07:50.880 --> 07:52.400]  They got a table back.
[07:52.400 --> 07:53.680]  Good thing I got my pretty table so
[07:53.680 --> 07:55.720]  I can see the difference between all of them.
[07:57.280 --> 07:59.800]  Okay, this is my biggest competitor by the way.
[07:59.800 --> 08:01.280]  They have so much money.
[08:01.280 --> 08:02.680]  I don't even know what to do with them.
[08:03.680 --> 08:04.400]  Beautiful soup.
[08:04.400 --> 08:06.600]  You get your regular HTML stuff.
[08:08.000 --> 08:09.960]  Very, very clean, but not really.
[08:11.480 --> 08:14.200]  Firecrawl, this is firecrawl.
[08:14.200 --> 08:19.720]  So for firecrawl you get a little bit better, you know, like very much
[08:19.720 --> 08:23.520]  markdown, like you got brackets and you got like links and stuff.
[08:23.520 --> 08:27.840]  So you can already see that you can already skip a bunch of stuff here.
[08:27.840 --> 08:30.400]  And if your large language model will love you,
[08:30.480 --> 08:31.800]  you give it clean data like this.
[08:32.840 --> 08:35.480]  But you take a look at Gina on this side.
[08:35.480 --> 08:37.840]  Gina is even more human readable.
[08:37.840 --> 08:41.240]  Even though they say it's supposed to bring you back markdown,
[08:41.240 --> 08:43.720]  they actually took away all the brackets and everything.
[08:43.720 --> 08:47.880]  So like you got bad and
[08:47.880 --> 08:51.760]  then you got markdown in like actual markdown format.
[08:51.760 --> 08:53.800]  And this one is promises you markdown but
[08:53.800 --> 08:57.200]  it's actually string in a human readable like format.
[08:57.200 --> 09:04.840]  And we can see that for pretty much every single examples here.
[09:06.440 --> 09:09.600]  The Gina one is usually the one that you can actually read.
[09:09.600 --> 09:14.480]  For example, this one from in the middle by Mendable.
[09:14.480 --> 09:16.920]  It's not actually human readable that much.
[09:16.920 --> 09:20.920]  Especially if your large language model doesn't care about URLs.
[09:20.920 --> 09:23.440]  Maybe you just wanted to know facts.
[09:23.440 --> 09:26.800]  Then you might not want all of this stuff in there, right?
[09:26.840 --> 09:30.360]  You might want just what's human readable.
[09:30.360 --> 09:32.200]  Maybe for a reasoning task, for example.
[09:34.280 --> 09:36.200]  Anybody have any questions so far?
[09:38.760 --> 09:42.800]  Okay, so that was surprisingly fast.
[09:42.800 --> 09:44.880]  All right, so we got also a cost table here.
[09:44.880 --> 09:46.800]  Let's take a look at this real quick.
[09:48.200 --> 09:50.000]  That's a great question, it reminds me of something.
[09:50.000 --> 09:54.040]  I think when I was playing with this, Adobe actually blocked BOS soup.
[09:54.040 --> 09:54.560]  So let me show you.
[09:57.000 --> 09:59.200]  What I'm gonna do is, I'm gonna just come with these two out.
[10:00.240 --> 10:05.240]  And then I wanna show more content and run this again.
[10:06.960 --> 10:11.800]  That's a great question, it reminds me of this.
[10:11.800 --> 10:12.480]  Okay, let's see.
[10:14.240 --> 10:14.840]  Articulate.
[10:16.600 --> 10:20.560]  Yeah, so this is like the most barebone, beautiful soup setup.
[10:20.560 --> 10:22.920]  Like this is like your intern doing it.
[10:22.920 --> 10:25.120]  This is not like very sophisticated.
[10:25.120 --> 10:27.440]  So you get four or three.
[10:27.440 --> 10:29.600]  That's why there was nothing there.
[10:29.600 --> 10:30.920]  So great catch, great catch.
[10:30.920 --> 10:32.560]  You can't compare the costs.
[10:32.560 --> 10:33.840]  Not enough for the first one.
[10:33.840 --> 10:37.400]  The first one was blocked, but actually I'm gonna run this again.
[10:38.400 --> 10:43.320]  But the table here, all we're doing is we're comparing the input cost
[10:43.320 --> 10:46.280]  between the three, if we were to put this into GPT-4.
[10:49.160 --> 10:51.880]  So let's just forget about the first one.
[10:52.840 --> 10:59.040]  This one, seven times, seven times less.
[11:00.160 --> 11:01.640]  And this one is even way less.
[11:03.040 --> 11:07.240]  I compare these two, 100 times, or 70 times.
[11:10.000 --> 11:10.720]  Yeah, any questions?
[11:10.720 --> 11:15.440]  So right now, these costs are just the cost of just the input tokens and
[11:15.440 --> 11:16.520]  output tokens, right?
[11:16.520 --> 11:18.200]  Just input tokens.
[11:18.200 --> 11:19.440]  For GPT-4.
[11:19.440 --> 11:20.520]  Yeah.
[11:20.520 --> 11:22.920]  But it has separate cost for input and output, right?
[11:22.920 --> 11:25.240]  It does, but this table alone, it's just input.
[11:25.240 --> 11:29.280]  Cuz the outputs, I'm gonna show the last step,
[11:29.280 --> 11:34.640]  which is using LM to extract our JSON of just the things that I want.
[11:34.640 --> 11:38.480]  Which is gonna be pricing tier names and the actual pricing.
[11:38.480 --> 11:41.440]  So the output's gonna be the same, regardless.
[11:42.560 --> 11:47.040]  But the input, if you use different tools, you get different amount of inputs.
[11:47.040 --> 11:54.080]  Yeah, especially this one, for example.
[11:54.080 --> 11:54.580]  Yeah.
[11:56.080 --> 11:56.840]  It's pretty nuts.
[11:56.840 --> 11:59.280]  I guess it's because of the extra tags that are there, right?
[11:59.280 --> 12:00.840]  In the fire crawl level.
[12:00.840 --> 12:05.880]  Yeah, especially, I think for this one, if you hit an image that has an entire
[12:05.880 --> 12:08.720]  binary on it, then you get the entire thing.
[12:09.880 --> 12:11.520]  This one also get the entire thing.
[12:11.520 --> 12:12.440]  Must have, right?
[12:12.440 --> 12:12.960]  Yeah.
[12:12.960 --> 12:16.800]  And then, I don't know what they're doing here.
[12:16.800 --> 12:18.840]  But they're doing some crazy stuff.
[12:20.160 --> 12:22.920]  And this is very clean, by the way.
[12:22.920 --> 12:23.840]  It's human readable.
[12:25.920 --> 12:29.720]  But again, if you want the actual URLs for
[12:29.720 --> 12:32.120]  most things, then you might be better off using this.
[12:33.160 --> 12:34.720]  Otherwise, you lose a lot of resolution.
[12:36.000 --> 12:40.280]  But again, if you want a reasoning task to be done, then you don't need that.
[12:40.280 --> 12:43.320]  You just need factual things.
[12:43.320 --> 12:48.080]  All right, so it's pretty obvious which one we should be paying for.
[12:49.600 --> 12:51.680]  Not knowing anything else about these companies.
[12:51.680 --> 12:58.520]  But now we're going to use OpenAI to kind of like try to do some extraction.
[12:58.520 --> 13:00.400]  Because I don't want to look at just the input.
[13:00.400 --> 13:05.800]  I want just one JSON with all the data that I was looking for in the beginning.
[13:05.800 --> 13:09.480]  Which is, which of these competitors having how many tiers and
[13:09.480 --> 13:11.560]  how much does it cost per each tier?
[13:11.560 --> 13:12.240]  That's all I want to know.
[13:14.040 --> 13:17.960]  Okay, so this part, all I'm doing is setting up an OpenAI client.
[13:19.120 --> 13:22.040]  And then I'm going to use my OpenAI key here.
[13:23.360 --> 13:25.920]  So we got a fresh key this week.
[13:27.920 --> 13:29.400]  Key from last week has been deprecated.
[13:31.080 --> 13:34.000]  And I'm using the latest and greatest GPT-4.
[13:34.000 --> 13:36.960]  Do you guys know what the O stands for?
[13:36.960 --> 13:38.400]  There you go.
[13:39.200 --> 13:41.320]  Do you guys know why in the demos,
[13:41.320 --> 13:45.400]  like GPT-4, or chat GPT sounds so flirtatious?
[13:45.400 --> 13:53.400]  Yeah, there's so many memes about that on Twitter.
[13:53.400 --> 13:58.280]  But okay, so this is just a utility function to then display
[13:58.280 --> 14:01.160]  the extracted content on another table.
[14:02.520 --> 14:07.240]  Because table and console is what we need to compare these things.
[14:08.160 --> 14:13.280]  All right, so what I'm doing here is just run GPT-4.0 through each of
[14:13.280 --> 14:16.680]  the inputs that we got before, like the entire input.
[14:16.680 --> 14:20.320]  So it could be like 50,000 tokens.
[14:20.320 --> 14:23.040]  Like it's not my money, it's Invest Ottawa's money.
[14:23.040 --> 14:26.760]  So I'm just running it through GPT-4.0 right now.
[14:26.760 --> 14:42.680]  Okay, so very, very simple entity extraction task using GPT-4.0.
[14:42.680 --> 14:47.760]  All I did was I put in a kind of like a chain,
[14:47.760 --> 14:51.880]  an LLM chain, and just be like, do, do, do, do, do, do.
[14:51.880 --> 14:56.480]  Give me the three pricing tiers from this website's content.
[14:57.160 --> 14:59.120]  So I give one website at a time.
[14:59.120 --> 15:01.920]  And then we turn a JSON with three keys.
[15:01.920 --> 15:06.400]  Cheapest, which is the cheapest tier, name of it, and then a price.
[15:06.400 --> 15:08.440]  And then I just give each one like a type, so
[15:08.440 --> 15:10.560]  that it knows what I'm actually looking for.
[15:11.600 --> 15:14.800]  And then the middle tier is the most expensive one.
[15:14.800 --> 15:17.480]  And that's it, that's my extraction tier.
[15:17.480 --> 15:22.280]  And always, what OpenAI does is it tells you that you should always,
[15:22.280 --> 15:25.440]  if you want JSON back, use type JSON object, but
[15:25.440 --> 15:27.640]  also say in your prompt that you want JSON back.
[15:27.640 --> 15:29.680]  So you always need those two things.
[15:30.920 --> 15:35.200]  But it's not really surprising that from BeautifulSuit we get nothing, so
[15:35.200 --> 15:37.640]  we can't extract anything here.
[15:37.640 --> 15:42.160]  Our JSON is completely zero and empty string,
[15:42.160 --> 15:47.760]  because we got 403 forbidden, we got forbidden from Adobe Articulate 360.
[15:49.240 --> 15:54.040]  Firecrawl seems like it's been able to get personal plan 0199,
[15:54.040 --> 16:01.120]  Teams plan 399, and then Reach 360 Pro, which is the enterprise tier.
[16:01.120 --> 16:04.240]  And the price is contact sales for pricing.
[16:04.240 --> 16:07.960]  This is weird because I gave it a type which was float, and
[16:07.960 --> 16:09.640]  it gave me a string here.
[16:09.640 --> 16:14.720]  It's still useful, but not technically accurate.
[16:16.080 --> 16:20.720]  And it comes out to GNI, price as variable.
[16:20.720 --> 16:25.200]  So they're kind of ignoring my instructions a little bit.
[16:25.200 --> 16:30.720]  Maybe I should have said float or no if it's not found or something like that.
[16:30.720 --> 16:35.000]  It just goes to show if you're working with large language models,
[16:35.000 --> 16:37.320]  unless you're working with dspy or something like that,
[16:37.320 --> 16:40.240]  you need to be very specific about your prompt.
[16:40.240 --> 16:42.480]  And just take care of most edge cases.
[16:43.160 --> 16:50.600]  So again, it seems like most of these tools pass the test.
[16:52.600 --> 16:54.720]  It's just a matter of whether or
[16:54.720 --> 16:57.760]  not you're gonna burn ten times the amount of money and
[16:57.760 --> 17:01.480]  do scraping manually using Beelzebub and your custom tools.
[17:01.480 --> 17:05.840]  Or you can kind of use one of these third party tools and just get
[17:05.840 --> 17:10.080]  clean markdown or clean user human readable text back.
[17:10.120 --> 17:14.160]  And just worry about your LLM stack instead of your web scraping.
[17:16.160 --> 17:20.080]  One last thing I wanna show you guys, which is scrape graph.
[17:21.600 --> 17:23.880]  So this is completely open source.
[17:23.880 --> 17:27.320]  This is not like the startups that we just saw earlier.
[17:27.320 --> 17:29.080]  This is completely open source and in Python.
[17:30.920 --> 17:34.360]  Anybody here familiar with graph data structure?
[17:36.000 --> 17:37.640]  I think we had a conversation before about this.
[17:37.640 --> 17:41.800]  It seems like you're a big fan of graph data structure too.
[17:41.800 --> 17:42.300]  Yeah.
[17:44.520 --> 17:47.760]  Okay, so open API key.
[17:49.560 --> 17:50.800]  And then link to script.
[17:50.800 --> 17:52.400]  Does anyone have a link that they wanna scrape?
[17:52.400 --> 17:55.000]  Which model you're gonna use?
[17:55.000 --> 17:56.160]  Okay, so this is what the website looks like.
[17:58.440 --> 17:59.240]  That's actually pretty cool.
[17:59.240 --> 18:02.440]  So what would be a question that you wanna ask?
[18:02.440 --> 18:06.320]  What I meant to say is, what do you wanna get out of this website?
[18:06.320 --> 18:09.160]  So model and speed graph.
[18:09.160 --> 18:09.660]  Yeah.
[18:11.160 --> 18:11.800]  What are these called?
[18:11.800 --> 18:13.840]  Electric unicycle.
[18:13.840 --> 18:14.360]  Okay.
[18:15.280 --> 18:15.960]  LA.
[18:15.960 --> 18:16.960]  EUC.
[18:16.960 --> 18:19.880]  And what color is the fastest model of the universe?
[18:19.880 --> 18:24.080]  Electric unicycle and.
[18:24.080 --> 18:24.880]  Model and weight.
[18:24.880 --> 18:26.960]  Model and name and speed.
[18:28.640 --> 18:29.140]  Okay.
[18:31.200 --> 18:33.520]  Yeah, initially I thought that you can just ask questions, but
[18:34.400 --> 18:38.680]  the prompt here is just what do you wanna get or scrape out of the website.
[18:38.680 --> 18:40.600]  Yeah, not like what do you wanna know by the website.
[18:42.040 --> 18:45.240]  Okay, so we got our answers back.
[18:45.240 --> 18:47.720]  So we got here, how do I get rid of this?
[18:49.840 --> 18:52.360]  Okay, so electric unicycle models.
[18:53.480 --> 18:55.480]  So you tell me if these models are legit.
[18:55.480 --> 18:56.280]  Yeah, yeah.
[18:56.280 --> 18:58.920]  I saw the names are correct and speed is correct.
[18:58.920 --> 19:00.360]  Which one's yours?
[19:00.360 --> 19:01.800]  It's the Gold Master.
[19:01.800 --> 19:02.960]  This one.
[19:03.480 --> 19:04.480]  50 plus miles per hour.
[19:04.480 --> 19:05.280]  Is that true?
[19:05.280 --> 19:05.920]  That's true.
[19:05.920 --> 19:07.120]  On full battery.
[19:07.120 --> 19:07.640]  Okay.
[19:07.640 --> 19:08.960]  Are you serious?
[19:08.960 --> 19:14.480]  You know, I drove like 72 kilometers per hour, my top speed, and it was scary.
[19:16.080 --> 19:20.480]  You see like all the, all everything like flying by you and think,
[19:20.480 --> 19:26.400]  if I'm gonna fall and hit something, it's not gonna, my body protection not gonna help.
[19:26.400 --> 19:27.000]  No.
[19:27.000 --> 19:27.480]  Wow.
[19:29.800 --> 19:31.440]  So that's pretty, yeah.
[19:31.440 --> 19:32.720]  Jesus.
[19:32.840 --> 19:33.880]  So this is accurate.
[19:33.880 --> 19:35.360]  That's accurate.
[19:35.360 --> 19:37.840]  V13, yeah.
[19:37.840 --> 19:39.160]  You know what?
[19:39.160 --> 19:44.600]  I actually, I actually went on their project and I asked, cuz I couldn't figure it out.
[19:44.600 --> 19:48.120]  Cuz I was trying to build this and I'm like, how many tokens is this consuming?
[19:48.120 --> 19:49.360]  Probably a lot.
[19:49.360 --> 19:51.080]  So I ask.
[19:51.080 --> 19:54.840]  And apparently there's like a function that you can do to get that.
[19:57.080 --> 19:58.080]  There's two ways.
[19:58.120 --> 20:03.360]  So if you use this library, just go to the discussions tab,
[20:03.360 --> 20:05.400]  look at my question and you'll find the answer.
[20:07.680 --> 20:09.480]  Yeah, I asked this like last night.
[20:09.480 --> 20:10.280]  I'm like, wait, wait.
[20:11.440 --> 20:13.760]  Are you trying to like hide this away from us or something?
[20:13.760 --> 20:16.520]  Like no, there's two ways to do it, but the documentation doesn't cover it, so.
[20:18.520 --> 20:19.800]  But yeah, that's all I got.
[20:19.800 --> 20:21.600]  Does anybody have any questions about anything?
"""

In [23]:
text_2 = """# tactiq.io free youtube transcript
# DevFest21 - AI/ML: Leveraging Prefabricated Pipelines for MLOps in Vertex AI
# https://www.youtube.com/watch/Hp7P99rsRpo

00:00:00.399 uh
00:00:01.360 speaker is lag
00:00:03.280 he going to talk about
00:00:05.839 the vortex ai which is newly released in
00:00:08.720 the i think a few months back in the
00:00:11.200 google i o
00:00:12.480 events it's
00:00:14.080 a
00:00:14.799 great platforms to unified many of the
00:00:18.480 resources for
00:00:20.160 machine learning and ai tasks
00:00:22.640 lag he is the director for that
00:00:26.080 analytics and ai solutions on google
00:00:28.480 cloud and his team builds software
00:00:31.279 solutions for business problems using
00:00:33.280 google cloud data analytics and machine
00:00:35.680 learning products
00:00:37.280 he founded the google
00:00:38.960 adwords solution lab machine learning
00:00:42.079 immersion program and it's also of
00:00:44.079 several oiling books and correlate
00:00:46.800 courses
00:00:48.399 without further ado let's welcome
00:00:50.800 lack
00:00:52.000 thank you bill thank hello everyone
00:00:55.039 so today i'm going to be talking about
00:00:57.600 machine learning pipelines specifically
00:01:00.760 pre-fabricated machine learning
00:01:02.399 pipelines in vertex ai
00:01:05.040 so what is vertex ai it's essentially a
00:01:08.080 set of tools for predictions for
00:01:10.159 training
00:01:11.280 for evaluation for monitoring so
00:01:14.479 essentially a set of tools that supports
00:01:16.479 your entire workflow
00:01:18.479 across different model types auto ml
00:01:21.200 tensorflow pytorch
00:01:23.280 scikit learn right whatever framework
00:01:25.200 you're using and different levels of ml
00:01:28.080 expertise whether you want to use low
00:01:30.479 code whether you want to use no code
00:01:32.880 with auto ml or you want to basically
00:01:35.600 use low code with bigquery ml or you
00:01:38.320 want to basically do sophisticated
00:01:41.200 programming with tensorflow or pytorch
00:01:43.680 it supports all of the different types
00:01:45.680 of ml expertise so what's in vertex ai
00:01:49.600 if you think about the machine learning
00:01:52.399 workflow on the top data readiness
00:01:55.119 getting your data ready labeling it for
00:01:57.280 example feature engineering right
00:01:59.920 creating a data set training a hyper
00:02:02.560 parameter tuning
00:02:04.079 then serving the model
00:02:06.240 understanding it tuning it deploying it
00:02:09.280 to the edge monitoring the model
00:02:11.840 managing the model
00:02:13.520 like there's like different
00:02:15.280 tools that support that full end-to-end
00:02:18.319 workflow and again as i said you could
00:02:21.360 do it with no code with automl for
00:02:23.680 things like vision video language
00:02:26.800 translation so when we talk about automl
00:02:29.120 what we're basically saying is using the
00:02:32.080 state-of-the-art models with uh you know
00:02:35.120 you know with that with a like point and
00:02:37.680 click interface but at the same time you
00:02:40.560 can also build things yourself so you
00:02:43.360 can you know build a train now you can
00:02:45.360 build things that can be trained hyper
00:02:47.040 parameter tune run experiments use gpus
00:02:50.959 do predictions explainability all of
00:02:53.440 these things
00:02:54.800 and as you build an ml pipeline right uh
00:02:57.920 you will want to orchestrate it you
00:02:59.599 don't want to basically do them one at a
00:03:02.080 time you want to connect them together
00:03:04.000 and that connection is what a pipeline
00:03:05.920 is and all of these are a unified
00:03:08.800 workflow so once you create a data set
00:03:12.480 you can use that same data set to train
00:03:14.879 both an automl model and a custom model
00:03:17.920 so you're not
00:03:19.200 doing completely separate workflows
00:03:22.560 just because you're changing the way
00:03:24.879 you're doing like for example if you
00:03:26.879 have things in a feature store you can
00:03:28.799 use it to support both pi torch and
00:03:31.120 tensorflow models that's the basic idea
00:03:33.840 is that all of these things are are
00:03:37.440 loosely connected and that you you can
00:03:39.840 combine them in the way that you want to
00:03:42.959 solve your problem
00:03:44.480 so the tools that you're using can be
00:03:47.040 like as i said user interface based low
00:03:50.159 code or it can be programmatic so
00:03:52.879 everything that you can do on the ui you
00:03:54.560 can also do
00:03:56.000 with a nice software development kit and
00:03:58.640 sdk
00:04:00.239 and of course today i'm going to be
00:04:01.840 focusing primarily on the right hand
00:04:04.000 side using the sdk to develop models
00:04:07.360 so but once you develop a model
00:04:09.760 that's not it right so
00:04:12.000 you develop the model you deploy it but
00:04:14.879 then you want to basically monitor it
00:04:17.358 retrain it update it and once you have
00:04:20.478 these kinds of complex
00:04:22.880 different steps that you want to do
00:04:25.440 automation becomes extremely important
00:04:27.520 and that's where pipelines come in so
00:04:29.680 when you think about a very simple view
00:04:32.160 of a pipeline it lets you do
00:04:34.759 experimentation try out different hyper
00:04:37.199 parameters try out different types of
00:04:39.919 models etc it allows you to do
00:04:43.280 training and later on to retrain the
00:04:45.680 model
00:04:46.639 it allows you to deploy the model and
00:04:49.120 then monitor it and you want to
00:04:51.440 basically do a variety of things during
00:04:54.160 that entire process and that's what
00:04:55.919 pipelines help you do
00:04:58.960 now how does vertex ai help there well
00:05:02.320 with all of the automation it helps you
00:05:04.400 do things like experiments try out
00:05:06.240 different things it helps you
00:05:08.720 reuse features it helps you do
00:05:10.560 continuous monitoring helps you manage
00:05:13.280 metadata throughout the entire process
00:05:15.919 but most of all it allows you to
00:05:18.160 orchestrate all of these things
00:05:20.600 automatically with caching with
00:05:23.440 dependency tracking in an ml pipeline
00:05:28.000 however
00:05:29.759 pipelines can be hard
00:05:32.400 and one of the things that
00:05:34.160 we firmly believe in
00:05:36.240 is that simple things should be simple
00:05:39.600 and hard things should be possible so
00:05:42.720 pipelines make hard things possible
00:05:45.680 right so when you talk about a pipeline
00:05:47.600 you're talking about hard things like av
00:05:50.720 testing performance monitoring uh uh
00:05:54.240 comparing different experiments etc so
00:05:56.560 that's great
00:05:58.000 it's great that lets you do these hard
00:05:59.840 things it lets you do feature stores
00:06:02.240 experiments continuous monitoring edge
00:06:04.639 deployment metadata no tracking
00:06:08.319 but what if you want to do something
00:06:09.680 simple
00:06:11.520 all i want to do is to develop a model
00:06:14.160 and in order to do that i want to
00:06:15.759 basically create a data set that i can
00:06:18.560 version that i know
00:06:20.400 what data i'm using to train my model i
00:06:22.639 want to train my model
00:06:24.479 and not just train one model i want to
00:06:26.560 tune the hyper parameters i want to also
00:06:29.360 try out auto ml so a little bit of
00:06:31.440 experimentation
00:06:33.039 deploy that model and get predictions
00:06:36.560 but this is the simple workflow
00:06:39.280 and i think
00:06:40.639 this workflow
00:06:42.479 is like 90
00:06:45.360 of what
00:06:46.800 you will do day to day right so
00:06:49.599 how do you do this simple workflow let's
00:06:51.599 forget all the complexities let's forget
00:06:53.840 all the sophisticated stuff how do you
00:06:56.080 do this simple workflow developing a
00:06:58.080 model training it tuning it automl
00:07:01.039 deployed to an endpoint get predictions
00:07:04.000 how do we do that in a simple way right
00:07:06.639 so let's start
00:07:08.240 first step is that you want to develop a
00:07:10.160 model this is the thing that every data
00:07:12.639 scientist like you know every tutorial
00:07:15.039 that's what it talks about right so you
00:07:16.960 go to a notebook you write some code and
00:07:20.400 in that code you can basically you know
00:07:22.960 have direct connections to things like
00:07:25.120 bigquery data proc spark
00:07:27.919 other things you want to be able to
00:07:29.840 maybe even schedule the notebook you
00:07:32.319 want to write the code in pytorch or you
00:07:34.960 want to write the code in tensorflow etc
00:07:37.360 so you do that in a notebook in in
00:07:40.560 vertex ai that's a vertex ai workbench
00:07:44.400 so inside the notebook then you
00:07:47.199 basically will write keras code that's
00:07:49.599 going to be my example so you will read
00:07:51.680 a data set create a training data set
00:07:53.840 you will create an evaluation data set
00:07:56.080 you will create a keras model you will
00:07:58.720 basically train the model
00:08:01.199 and you will save the model this is your
00:08:03.599 normal
00:08:04.800 keras workflow
00:08:06.560 and like let's just assume that that's
00:08:08.639 basically what we're going to do so what
00:08:10.479 i'm doing here is that i'm just opening
00:08:12.400 up jupiter lab right right from my
00:08:15.759 vertex ai and at this point
00:08:18.800 i'm basically in the jupyter ui i can
00:08:21.440 get clone the repo here
00:08:23.759 right but let me just go ahead and do it
00:08:25.759 from the command line and let me do git
00:08:28.639 clone
00:08:30.960 github.com
00:08:34.958 um and
00:08:36.799 i'll do the data science on gcp
00:08:40.880 okay
00:08:42.000 and
00:08:43.039 move into that
00:08:44.800 let me move to the branch edition too
00:08:48.640 right so there we are
00:08:50.640 and in here
00:08:52.480 is
00:08:53.519 my notebook
00:08:54.959 right so i can basically go ahead and
00:08:57.680 say that i want to basically use python
00:08:59.680 3 right now i'm in my notebook and this
00:09:03.519 is should be pretty familiar to anybody
00:09:05.680 right so if i want right i basically out
00:09:08.800 here i'm basically saying that i want to
00:09:10.160 train my notebook on us central one
00:09:12.880 right this is my bucket this is my
00:09:15.120 project and here i'm running a bigquery
00:09:18.800 query to basically go pull some data do
00:09:22.399 some data preparation and create a brand
00:09:25.200 new table called training data and a
00:09:27.920 brand another new table called
00:09:29.680 evaluation data right and then
00:09:32.640 having done that i'm basically going
00:09:35.200 ahead and extracting it out into cloud
00:09:38.320 storage i can read the data directly
00:09:40.800 from bigquery there's a bigquery reader
00:09:42.800 in tensorflow but it turns out that if
00:09:45.600 you have it in
00:09:47.440 cloud storage it it gives me a few other
00:09:50.080 benefits that again i i go through later
00:09:53.279 but let's not worry about it so in this
00:09:54.959 case rather than training directly of
00:09:57.360 bigquery i'm actually extracting it out
00:09:59.360 into a csv file and i basically get
00:10:02.000 three csv files evaluation training and
00:10:05.760 all
00:10:06.720 and basically this is all just normal
00:10:10.079 code i'm using tf data
00:10:12.399 reading the data set making sure the
00:10:14.720 data is okay
00:10:16.000 then going ahead and creating my
00:10:19.279 wide and deep model doing feature
00:10:21.519 engineering on it training the model
00:10:24.640 and now i have my model this is my
00:10:27.600 entire keras model with a whole bunch of
00:10:29.920 inputs with a few dense layers
00:10:32.480 and
00:10:33.360 having done this
00:10:34.959 i now train my model i have my model
00:10:38.480 it all works so this is basically the
00:10:40.959 first part your part of your data
00:10:42.640 science workflow and then if you want
00:10:45.360 you can go ahead and take that model and
00:10:47.279 deploy it using gcloud right so you can
00:10:50.160 deploy the model using gcloud
00:10:52.560 into
00:10:53.680 vertex ai so
00:10:56.000 that's one way this this would work
00:10:58.399 and then i can take this entire
00:11:01.440 notebook
00:11:02.880 and in vertex ai i can basically
00:11:04.880 schedule the notebook i can say okay i
00:11:07.360 would like to go to this notebook
00:11:09.519 and i would like to like schedule it
00:11:12.480 right so i would like to run a scheduled
00:11:14.240 run of this notebook at certain times
00:11:17.120 and then because this notebook does
00:11:19.279 everything from
00:11:20.959 data preparation
00:11:22.959 all the way to model deployment
00:11:26.399 i have everything ready and it will
00:11:29.440 basically
00:11:30.720 uh be be a runnable thing
00:11:33.519 so that's obviously one way that i could
00:11:35.360 do this
00:11:36.480 but it doesn't really support the kind
00:11:38.800 of orchestration and
00:11:40.720 everything that i want to do i want to
00:11:42.079 automate it right so what do i so one of
00:11:44.800 the things that i really
00:11:47.200 believe
00:11:48.240 is that notebooks are great for
00:11:49.760 development
00:11:51.360 um and you can use the notebook as a
00:11:53.760 step of a pipeline you can schedule the
00:11:55.839 entire notebook but really
00:11:58.480 right for production purposes
00:12:01.360 put your code in python files right
00:12:03.680 because you can basically manage them
00:12:05.680 and maintain them using your traditional
00:12:08.639 software workflows they like notebooks
00:12:11.519 are great for data scientists but for ml
00:12:14.399 ops for operations for maintainability
00:12:18.160 nothing beats having flat files having
00:12:21.120 python files so what i've also then done
00:12:24.399 once i've developed the notebook
00:12:26.560 right what i did was i basically took
00:12:28.880 all of the code in this notebook
00:12:31.120 and moved it into a file called
00:12:32.959 model.pipe
00:12:34.480 right it's pretty easy to do you can
00:12:36.240 basically go into your notebook and you
00:12:38.639 can basically export the notebook as a
00:12:41.440 uh
00:12:42.240 not exported yeah save the notebook as a
00:12:44.480 python file right and then basically
00:12:47.360 remove all of the display stuff and you
00:12:49.839 basically have it so all of this code is
00:12:52.399 essentially the code that was in that
00:12:54.959 notebook
00:12:56.160 but it's essentially now callable
00:12:58.880 and i added in
00:13:00.480 a main right so that i could basically
00:13:02.639 call it with a bunch of parameters like
00:13:05.920 specifying the bucket specifying their
00:13:08.399 training batch size etc so once i have
00:13:12.160 that notebook
00:13:14.160 right and i've basically gotten this we
00:13:16.800 can now go ahead and try it out so this
00:13:19.839 model here
00:13:21.279 is is now
00:13:23.200 workable
00:13:24.720 and i can basically go ahead and call it
00:13:28.240 right with uh now python3 model.5 and i
00:13:33.040 have a like you know thing called minus
00:13:34.880 minus developed and i can go ahead and
00:13:37.040 run this make sure that all of this
00:13:39.040 works so great
00:13:41.199 so at this point i have my code
00:13:44.000 but
00:13:44.720 remember that in this notebook i was
00:13:47.519 doing some weird stuff
00:13:49.600 i was also calling gcloud commands to
00:13:52.399 deploy the model
00:13:53.839 this is not python anymore this is bash
00:13:57.920 how do i automate these
00:14:00.079 well you could do it in bash but that's
00:14:02.560 not what i would recommend keep things
00:14:04.639 in python keep this whole automation
00:14:07.120 pipeline in python so that's basically
00:14:09.680 what i'm going to be talking about here
00:14:11.920 so
00:14:13.120 remember the the simple thing here that
00:14:15.360 we're talking about right we're
00:14:17.279 basically saying we want to develop a
00:14:18.959 model but having developed the model
00:14:21.360 let's look at this this entire workflow
00:14:23.839 creating the data set training a model
00:14:26.160 deploying to an endpoint getting
00:14:27.839 predictions
00:14:28.959 how to do that in python so that you can
00:14:31.440 automate it really well so this is
00:14:33.519 basically the workflow so you've now
00:14:36.560 taken the model you've saved it
00:14:38.880 now you can basically upload the model
00:14:41.920 as a vertex ai model
00:14:44.399 and then deploy the model to an endpoint
00:14:47.839 in such a way that when a client request
00:14:50.639 comes in through json
00:14:52.639 this endpoint basically invokes your
00:14:55.040 model
00:14:56.000 and sends back a json payload and the
00:14:58.639 endpoint doesn't contain just one model
00:15:01.839 the end point can contain multiple
00:15:04.079 models and do a traffic split between
00:15:06.720 them
00:15:07.600 so that's the basic basic idea so the
00:15:10.320 concepts here are you have your model
00:15:12.880 just model files you upload them into a
00:15:15.440 model object
00:15:16.880 you put model objects into endpoints and
00:15:20.000 you basically do traffic splits in the
00:15:22.639 endpoint between different models so
00:15:25.120 let's start with the first thing you
00:15:27.199 want to train a model you want to have a
00:15:28.959 data set
00:15:30.560 so the reason that you want to have a
00:15:32.320 data set that is in vertex ai
00:15:35.040 we think of four different types of
00:15:37.279 objects
00:15:38.480 a data set which could be structured or
00:15:40.959 unstructured it will have metadata it
00:15:43.600 will have versions so that you know
00:15:46.079 given a model what version of data was
00:15:48.800 used to train that model that's part of
00:15:50.560 the metadata of that model
00:15:52.880 the next thing that you will often have
00:15:54.639 is that you'll have the model itself you
00:15:56.399 trained a model now you have model files
00:15:58.880 your saved model in tensorflow right you
00:16:01.360 have machine learning model artifact so
00:16:04.079 those files that's your model object and
00:16:07.440 the end point is the thing that gets
00:16:09.440 called it contains multiple models and
00:16:12.240 you orchestrate all of these things
00:16:14.800 using a training job or a pipeline right
00:16:17.759 and the pipeline basically is take a
00:16:20.320 data set train a model on it deploy to
00:16:22.560 the endpoint
00:16:24.320 so essentially the way you start is that
00:16:27.279 you basically have now in my case a
00:16:29.600 train on vertex ai right and i basically
00:16:32.720 initialize the ai platform
00:16:35.199 and i say
00:16:36.399 my data set
00:16:38.000 is a tabular data set
00:16:40.320 it consists of all of the files that
00:16:43.600 meet this wild card thing so i'm
00:16:46.000 basically using tensorflow right g
00:16:48.839 file.glob find all of the all star.csv
00:16:53.279 and using those files to create my
00:16:56.560 tabular data set so now this data set
00:16:59.680 contains a number of metadata
00:17:02.079 information that is going to get passed
00:17:04.319 along i don't you will not actually see
00:17:06.559 any metadata in any of my code but
00:17:08.959 remember that underneath
00:17:10.799 vertex ai is taking care of all of that
00:17:12.720 for us okay so we're basically getting a
00:17:15.679 tabular data set
00:17:17.599 and it consists of all of these files so
00:17:20.319 that's pretty much it
00:17:21.839 so then in model.pi
00:17:25.919 earlier we kind of knew what we were
00:17:28.319 reading but now
00:17:30.320 vertex ai is going to basically set a
00:17:32.320 few environment variables for us
00:17:34.720 it's going to tell us what the training
00:17:37.520 data is
00:17:38.720 what the evaluation data is and what the
00:17:40.880 test data is
00:17:42.480 those are three of our true environment
00:17:44.320 variables
00:17:45.440 and it's going to tell us
00:17:47.280 where to save the output
00:17:49.760 okay so what i will do
00:17:51.919 is that in my model.pi
00:17:54.400 i will look for this environment data
00:17:56.960 and if it's not present then i know that
00:17:59.919 somebody's running it not in the context
00:18:02.320 of a pipeline where these things are set
00:18:05.440 but they're probably running it from a
00:18:07.360 notebook they're probably running it
00:18:09.679 locally right and so you can basically
00:18:12.240 say if that's not set
00:18:14.559 like
00:18:15.360 hardcoded to be where we expect it to be
00:18:18.160 okay so once you do that you now have
00:18:21.120 your dataset created and now you're able
00:18:24.000 to train your model so
00:18:26.080 the training the model means
00:18:28.559 that you have your model code that's
00:18:31.120 your custom training job and you're
00:18:33.440 saying go ahead and please run
00:18:35.120 model.pipe
00:18:37.600 that's your code
00:18:39.520 but in order to run it we will need a
00:18:42.799 container vertex ai works on containers
00:18:45.200 everything is containerized
00:18:47.120 fortunately
00:18:48.559 there is a pre-built container
00:18:51.200 for every version of tensorflow that you
00:18:53.200 want to use
00:18:54.320 so in this case i'm basically saying
00:18:56.880 like i want to train on the gpu version
00:18:59.360 of tensorflow and i want to deploy on
00:19:01.840 the cpu version of tensorflow right so
00:19:04.640 basically i'm getting two
00:19:06.720 training image and the deployment image
00:19:09.039 that's the training image this is the
00:19:10.480 prediction image i have two two
00:19:13.120 pre-built containers and so when i'm
00:19:16.000 when i'm training it i'm saying use this
00:19:18.559 as your training image and serve it on
00:19:20.880 the deploy image so pretty much
00:19:23.600 i'm can take my model.pi and use it to
00:19:26.880 create a custom training job and once
00:19:29.679 i've created a custom training job i run
00:19:32.559 the job
00:19:34.000 passing in the data set
00:19:36.160 right and passing in the accelerator
00:19:39.039 type the machine type etc that i want to
00:19:41.919 do the training on
00:19:43.760 so bottom line then you train your model
00:19:46.480 you create all the training code and
00:19:48.559 then when you're ready to train the
00:19:49.919 model on some piece of hardware what you
00:19:52.640 do is you create a custom training job
00:19:55.039 providing the tensorflow containers this
00:19:57.039 is where the prefab
00:19:58.720 part of my title comes in it's all
00:20:01.200 already there prefabricated so just go
00:20:04.159 ahead and use the pre-built container
00:20:07.200 pass in the hook for your custom code
00:20:09.760 and remember that your custom code right
00:20:12.480 will basically you this is the contract
00:20:15.200 that your custom code has to honor it
00:20:17.679 has to read the data from the path
00:20:20.799 pointed by these three environment
00:20:22.559 variables and write the data to the path
00:20:25.520 pointed by this thing so if you look in
00:20:28.000 my code in model.pi
00:20:30.400 right so basically that's what i'm doing
00:20:33.039 i'm basically setting the output there
00:20:35.280 based on the output model there i'm
00:20:37.760 setting the training data pattern based
00:20:39.919 on the environment variable right and if
00:20:42.240 the environment variable isn't set
00:20:44.320 i'm basically dropping back to the
00:20:47.360 original behavior i had in my notebook
00:20:50.000 right my original notebook was basically
00:20:52.880 reading train star so i just keep that
00:20:55.840 if
00:20:57.840 i'm not running in a pipeline if i'm
00:20:59.440 running in a pipeline i'm getting the
00:21:02.240 the orig the the place to read from
00:21:05.440 from the environment variable
00:21:07.360 the rest of my code remains exactly the
00:21:10.400 same right because all of this is
00:21:12.720 basically just processing the training
00:21:14.880 data pattern evaluation data pattern and
00:21:17.840 that comes either from the environment
00:21:19.520 variable or from a hard-coded value
00:21:21.919 because i'm calling it from a notebook
00:21:23.919 right so given those two things
00:21:27.200 at this point
00:21:28.720 i now have my
00:21:30.480 model training done
00:21:32.240 okay
00:21:33.039 how does this change if you're not doing
00:21:35.200 a custom model remember that this is if
00:21:37.120 you are writing tensorflow code myself
00:21:40.159 and i'm saying i want to run it here
00:21:42.000 well if you're doing auto ml we will
00:21:44.559 look at it so we basically change this
00:21:47.039 function train custom model to be train
00:21:50.159 auto ml model this part will change but
00:21:53.039 the rest of my pipeline is going to
00:21:55.120 remain the same i'm going to continue to
00:21:57.440 use the same data set
00:22:00.080 but i will use that same data set
00:22:02.960 for training a vertex ai model for
00:22:05.440 training an automl model for doing hyper
00:22:08.159 parameter tuning and then regardless of
00:22:10.480 how i got my model i'll deploy it in
00:22:13.360 exactly the same way so this is part of
00:22:15.679 that unification that's super important
00:22:18.080 it doesn't matter
00:22:19.840 how you train your model this could be a
00:22:21.679 pie torch model
00:22:23.280 and you would still create the data set
00:22:24.960 and you would still deploy it in exactly
00:22:26.640 the same way that's part of the power of
00:22:28.559 this you can basically switch any of
00:22:31.200 these things in the middle and you have
00:22:32.640 that unified workflow making life a lot
00:22:35.200 easier
00:22:36.320 so what if you're not using these
00:22:37.919 pre-built containers that i did that i
00:22:39.840 showed you with uh with tensorflow well
00:22:42.720 there's two other two other ways that
00:22:44.400 you could do it you could use tube flow
00:22:47.200 pipelines and you could basically write
00:22:50.000 whatever code you want
00:22:52.000 right and basically have a base image
00:22:55.039 and
00:22:55.840 now create your own docker image write
00:22:58.240 your own code and use that to basically
00:23:01.760 create a custom training job
00:23:04.080 and that custom training job is
00:23:05.840 basically the stuff in the middle here
00:23:07.679 right you need this custom training job
00:23:10.640 and you just do a job at run
00:23:12.559 everything remains the same you've just
00:23:14.320 gotten a new custom training job out of
00:23:17.039 your own container out of your own code
00:23:19.760 so that's one way to do it
00:23:21.520 the other way to do it is to say forget
00:23:23.360 about all this prefab
00:23:25.039 pipelines business i will go use tfx and
00:23:28.559 tfx is up is a very prescriptive way of
00:23:31.440 running things where basically provides
00:23:34.000 you
00:23:35.039 python libraries for data validation
00:23:37.520 python libraries for a tensorflow
00:23:40.240 transformation etc you use all of those
00:23:43.520 tfx components
00:23:45.520 and you use it to create
00:23:47.600 a pipeline object and you run that
00:23:49.919 pipeline object okay so you have
00:23:52.080 multiple ways of creating pipelines i'm
00:23:54.640 not going to talk about these two two
00:23:56.240 other approaches because today i'm
00:23:58.480 talking about the very like prefab
00:24:01.520 simple things should be simple approach
00:24:03.840 this is the more
00:24:05.200 you can do hard things and hard things
00:24:06.799 are possible approach
00:24:09.120 so once you've gotten your model
00:24:12.080 what you do to your model you deploy to
00:24:13.840 an endpoint
00:24:15.039 so you want to create an endpoint
00:24:17.679 now so here's the thing though
00:24:19.520 you don't want to create an end point if
00:24:21.520 it already exists remember the idea
00:24:23.440 behind an end point you want to do
00:24:24.880 traffic splits
00:24:26.320 if every model gets deployed to a new
00:24:28.640 endpoint
00:24:30.320 it doesn't make sense so you want to
00:24:32.240 keep the end point consistent and you
00:24:34.720 want to keep deploying more and more
00:24:36.320 models into the same endpoint and
00:24:38.799 splitting traffic between
00:24:41.520 different trained models
00:24:44.159 as you go along so that's what i'm doing
00:24:46.240 here i'm saying go ahead and get me all
00:24:48.960 the end points with a specific display
00:24:51.279 name
00:24:52.159 and order it by the descending order of
00:24:54.960 create time and take the most recently
00:24:57.919 created endpoint
00:24:59.760 and that's the end point that i'm going
00:25:01.520 to deploy ideally you've only created an
00:25:04.080 endpoint once
00:25:05.840 and then from then on you're just
00:25:07.679 reusing it over and over and over again
00:25:09.760 right so create the endpoint with a
00:25:11.360 specific name this endpoint name
00:25:14.400 and reuse it over again
00:25:16.480 so once you do that
00:25:18.159 you can basically say model.deploy
00:25:20.400 passing in the endpoint passing in the
00:25:22.960 traffic split so here i'm saying this
00:25:25.440 model that i'm deploying
00:25:27.600 given 100 of the traffic
00:25:29.919 if you want you can basically pass in
00:25:32.320 previously deployed model ids
00:25:34.960 and say that model has 30 and i get 70
00:25:39.440 or vice versa specify the machine type
00:25:42.320 that you want to deploy on
00:25:44.159 and that's pretty much it right
00:25:47.679 so once you have deployed the model to
00:25:50.159 an endpoint you want to use it
00:25:53.120 so the way that you can use it is you
00:25:55.360 have to basically create a json request
00:25:58.240 so this is what the request looks like
00:26:00.000 these are all of the input features to
00:26:02.000 the model so you basically go through
00:26:04.080 and you pass it you create a json
00:26:05.840 something and you pass that in
00:26:07.840 and
00:26:08.720 you basically say endpoints predict and
00:26:11.760 pass in your json request so this is how
00:26:14.240 you can do it
00:26:15.520 with bash just to try things out during
00:26:17.919 development
00:26:19.120 but obviously your client code is not
00:26:21.440 going to use gcloud
00:26:23.120 your client code wants to use python
00:26:25.600 wants to use rest wants to use java
00:26:28.640 wants to use something so if you want to
00:26:30.720 use python
00:26:32.240 you basically get the end point as
00:26:34.240 before
00:26:35.360 and then you basically create a no
00:26:38.559 python dictionary with the exact same
00:26:41.279 json structure
00:26:43.039 and you pass in
00:26:45.039 endpoint.predict
00:26:46.720 you pass in the data and you get the set
00:26:49.840 of predictions
00:26:51.120 okay so that's pretty much uh how you do
00:26:53.679 it and you can also do it using rest you
00:26:56.400 can basically use curl and you can pass
00:26:58.640 in a specific thing so there's
00:27:01.279 it's just a rest api so you can invoke
00:27:03.520 it as long as you basically put your
00:27:05.520 data in this form and you post it you're
00:27:07.919 fine
00:27:09.520 so now let's look at a couple of the
00:27:11.600 other like
00:27:12.960 how does it change if i wanted to tune
00:27:14.880 hyper parameters
00:27:16.480 well if you want to tune hyper
00:27:17.760 parameters
00:27:19.120 inside your model.pi you've got to do
00:27:21.279 two things
00:27:22.640 first thing
00:27:24.000 is anything that you want to tune has to
00:27:27.360 be a command line parameter to your
00:27:29.360 model so if you want to tune the number
00:27:31.440 of buckets that you're going to
00:27:33.120 discretize your variable into you make
00:27:36.000 that an input parameter into your into
00:27:38.720 your
00:27:39.520 code
00:27:40.559 and then you basically
00:27:42.559 write
00:27:44.320 a keras callback so this is a this is my
00:27:47.200 hyper parameter tuning callback in keras
00:27:50.240 i say at the end of every epoch
00:27:53.360 please report
00:27:55.440 the metric that i care about which is
00:27:57.600 the
00:27:58.320 validation root mean squared error so
00:28:01.279 that is my hyperparameter metric
00:28:03.679 and say report that
00:28:05.760 and then create the model train it then
00:28:08.240 as you normally do but when you call the
00:28:10.799 model that fit remember to also call it
00:28:14.080 pass in the hyper parameter tuning
00:28:16.320 callback
00:28:17.600 that's all you need to do
00:28:19.200 now your keras model is going to be
00:28:21.200 reporting metrics as it goes along
00:28:24.880 and then in
00:28:26.080 in their training pipeline code
00:28:29.279 you built the original job that you had
00:28:31.760 the custom training job
00:28:34.000 it is now just a single trial so you say
00:28:37.120 give me a trial job
00:28:39.360 from my model.script
00:28:41.600 and you use that to create a hyper
00:28:43.600 parameter tuning job
00:28:45.840 and you train that job
00:28:48.240 and the result of it is going to be a
00:28:50.640 number of trials
00:28:52.399 you sort it find the best trial and
00:28:55.760 using the best set of parameters you can
00:28:58.399 basically go train it again right to
00:29:00.960 train it all the way through typically
00:29:03.039 what you do when you do hyper private
00:29:04.960 tuning is that you train on a smaller
00:29:07.520 set of data faster
00:29:09.919 and then like once you've found the best
00:29:12.159 set of parameters you tune that
00:29:15.200 you take those parameters and you train
00:29:17.840 the mod the model on the full data set
00:29:20.480 for much longer so that's basically what
00:29:23.279 this is showing here so if you go into
00:29:27.919 train on vertex ai dot pi
00:29:30.320 the hyperbrain retuning job is to
00:29:32.799 basically create my custom job for my
00:29:35.919 local script passing in the model.pi
00:29:39.520 saying please don't evaluate on the full
00:29:41.919 data set just train on a smaller number
00:29:44.640 of examples
00:29:46.159 and go run this code for me that's my
00:29:48.720 trial job and then my hyper parameter
00:29:51.360 tuning job
00:29:52.720 basically says i want to minimize my
00:29:55.039 validation rmse and these are the three
00:29:58.159 parameters i want you to tune
00:30:00.640 the training back size the number of
00:30:02.720 buckets and the number of layers and
00:30:04.559 nodes in my
00:30:05.919 in my dnn model and run it for these
00:30:09.520 many number of trials
00:30:11.520 and then once you run it you get all of
00:30:14.159 your trials you find the best one
00:30:16.960 and you go through all of those find all
00:30:19.360 of the best parameters and you basically
00:30:22.159 go ahead and call the original
00:30:24.880 train custom model the one that we first
00:30:27.120 wrote with the full set so that's pretty
00:30:30.080 much
00:30:30.799 hyper grammar tuning
00:30:32.960 how do you do auto ml well automl means
00:30:36.320 instead of training a custom job you're
00:30:39.200 going to use an automl tabular training
00:30:41.440 job and run it so here
00:30:44.240 right uh the
00:30:46.240 train that's a custom model here's my
00:30:49.039 trained automl model i'm creating a job
00:30:52.640 i'm doing job.run i'm returning the
00:30:54.960 model
00:30:56.000 so regardless of how i did it i trained
00:30:59.279 my data set
00:31:00.799 i did one of the three training jobs
00:31:03.519 automl or hyperparameter tuning are
00:31:06.240 custom modeling
00:31:07.760 and then i deploy to the endpoint
00:31:10.559 and i'm done
00:31:12.080 right i'm done at this point and i can
00:31:14.720 take this entire code my main
00:31:17.679 and i can basically run that entire
00:31:19.760 thing as a pipeline to basically get all
00:31:22.559 my metadata tracking and everything
00:31:24.399 going
00:31:25.200 okay so
00:31:27.440 bottom line at this point what you've
00:31:28.799 done is you've created this workflow you
00:31:31.120 basically take your model in your keras
00:31:32.799 code you've done a model that's safe you
00:31:34.559 upload the model right you deploy the
00:31:37.120 model and you make it available for
00:31:38.880 predictions
00:31:40.480 so
00:31:41.840 to summarize
00:31:43.279 prefab training pipelines they make
00:31:45.440 simple workflows really simple
00:31:47.120 developing a model creating a data set
00:31:50.159 training it tuning it deploying the end
00:31:53.279 point and getting predictions
00:31:56.240 for if you want to do custom or
00:31:57.760 continuous evaluation for example all
00:32:00.399 that you need to do is when you do the
00:32:02.559 job.run
00:32:04.000 also export the evaluations into a
00:32:06.640 bigquery table and
00:32:08.880 provide that bigquery table to vertex ai
00:32:11.519 and it will automatically start
00:32:13.120 monitoring your metric and show you
00:32:16.080 graphs of it and then you can set up
00:32:18.320 when this
00:32:19.360 metric drops below a certain point
00:32:22.320 retrain right that's pretty much it so
00:32:24.960 it's it's very very very straightforward
00:32:27.760 to start to add many of these amazing
00:32:30.240 capabilities to this very simple
00:32:33.519 starter point
00:32:35.200 so in summary right we talked about uh
00:32:38.480 this simple workflow like follow me on
00:32:40.880 twitter uh lac underscore gcp
00:32:44.000 uh this
00:32:45.279 uh
00:32:46.159 i i will leave here um
00:32:49.519 oh
00:32:50.640 okay the uh the github repository which
00:32:53.760 is probably the most important thing
00:32:55.120 here uh is this guy
00:32:57.760 okay
00:32:59.279 so go there
00:33:01.360 uh just one
00:33:03.200 warning
00:33:04.799 make sure to switch to edition two right
00:33:07.679 because uh you know
00:33:09.519 so right there
00:33:10.880 there is o9 vertex ai so just remember
00:33:14.159 this url i'll also post this in the chat
00:33:22.720 all right and with that i'll take
00:33:23.919 questions
00:33:25.760 awesome great uh wow the vertex ai
00:33:28.880 really makes the machine learning tasks
00:33:31.360 easy and
00:33:32.180 [Music]
00:33:33.440 the full stack
00:33:35.279 um yeah we have
00:33:37.120 quite a lot of questions uh let me bring
00:33:40.640 up uh the sli uh sign up for that
00:33:44.240 um just a quick reminder
00:33:47.679 right off of this right and uh yeah so i
00:33:51.679 just posted that and uh if you go to
00:33:54.399 github there is the ipython notebook
00:33:56.960 okay right right right great yeah
00:33:59.279 yeah i just wanted to remind that you
00:34:00.960 know for anybody for uh questions
00:34:03.760 uh sorry just a quick reminder you know
00:34:05.760 if you have questions uh you can post in
00:34:07.679 the chat or raise hand we we can uh um
00:34:11.280 build you to speak
00:34:12.879 uh let's go with
00:34:14.399 the i have a few first questions
00:34:17.918 um one question is is there any further
00:34:20.719 granule level control on where my
00:34:23.679 endpoint will be deployed within a
00:34:26.000 region
00:34:27.119 uh
00:34:27.918 you can do it within a zone i would not
00:34:30.320 i would not uh go there i mean again a
00:34:33.280 region in google cloud is probably the
00:34:35.839 most granular that you want to go right
00:34:38.000 so so specify a specific region i mean
00:34:40.879 why would you ever want to basically
00:34:42.399 specify a single zone that doesn't make
00:34:44.879 too much of sense right so so stick to a
00:34:46.879 region a region is probably the most
00:34:48.800 granular that you want that you want to
00:34:50.560 be you might want to do a multi-region
00:34:52.879 you might want to do for like you know
00:34:55.760 resilience purposes but a region is the
00:34:58.800 is the lowest ground
00:35:02.720 okay great another question from davis
00:35:05.680 why are you using jupiter lab so the
00:35:08.400 collab is google's version of adobe to
00:35:10.880 lab
00:35:12.480 that's the absolute colab is our free
00:35:15.760 version we have now for some people who
00:35:18.480 wanted to use it in enterprise we now
00:35:20.320 have a paid version of colab but uh in
00:35:24.240 vertex ai what we call vertex ai
00:35:26.320 workbench uh is basically our
00:35:29.040 development environment that provides
00:35:31.680 enterprise capabilities like vpc service
00:35:34.400 controls encryption uh like you know
00:35:37.880 multi-tenancy a whole bunch of things
00:35:40.000 that if you're running it in an
00:35:41.680 enterprise if you have confidential data
00:35:44.240 i strongly suggest using the vertex ai
00:35:46.960 workbench
00:35:50.800 another question is asking is the
00:35:52.720 voltage ai free like a collab or
00:35:56.240 different vertex is part of google cloud
00:35:59.040 and you will be a google you know it is
00:36:01.200 definitely definitely a a paid service
00:36:04.160 it's not free okay uh there is a free
00:36:06.880 trial version of google cloud where you
00:36:08.640 get 300 free so definitely use it to
00:36:11.920 learn but now when you're using in
00:36:14.000 production you you know it's not free
00:36:19.440 all right another question
00:36:21.760 uh
00:36:23.839 to clarify the question from chester to
00:36:26.240 clarify are you claiming that vertex ai
00:36:28.640 can accomplish what a spark plus h2o
00:36:32.640 plus ml flow plus ml1 frameworks will do
00:36:38.800 spark is larger than vertex ai right
00:36:42.400 spark is a combination of data
00:36:44.640 processing
00:36:46.000 and machine learning so if you're
00:36:47.839 talking about spark m light then yes
00:36:50.079 right but remember that spark also helps
00:36:52.720 you do data preparation so on google
00:36:56.400 cloud you would run spark within data
00:36:59.119 proc right so you could use data proc to
00:37:01.680 prepare your data the way you do like
00:37:03.920 large data sets with spark more commonly
00:37:06.640 on google cloud people use bigquery to
00:37:08.800 prepare data but you can do spark or you
00:37:11.119 can do bigquery to prepare the data but
00:37:13.440 once that data has been prepared as far
00:37:16.960 as feature engineering and training and
00:37:20.640 you know no code no code all of these
00:37:23.200 different frameworks yeah that unified
00:37:25.200 framework unified orchestration you're
00:37:28.560 scheduling all of these things
00:37:30.240 absolutely don't
00:37:32.079 don't buy individual things and glue
00:37:34.240 them together buy something that is
00:37:36.160 already well integrated would be my
00:37:37.760 thing yes i am i am making the claim
00:37:40.960 that other than data pre-processing
00:37:42.880 which is not which is not
00:37:45.200 but everything machine learning related
00:37:46.720 you can do in wordpress app bigquery
00:37:48.960 plus vertex ai is an extremely powerful
00:37:51.119 solution as is data proc plus vertex
00:37:57.280 great
00:37:58.160 use it for free while learning it yes uh
00:38:01.200 again a google cloud offers a lot of
00:38:03.520 things free for students so go to
00:38:06.800 cloud.google.com
00:38:08.720 edu for example right and you will get a
00:38:12.480 lot of higher education programs have
00:38:14.880 your faculty members if you're a student
00:38:17.520 register for it so that all of your
00:38:19.359 class can get access to it strongly
00:38:21.760 encourage you to do that if you also go
00:38:23.920 to cloud.google.com there is a 300 free
00:38:26.560 trial that you can use to learn as well
00:38:30.240 the third way to learn relatively
00:38:33.040 quickly is quicklabs.com
00:38:36.079 quick labs has a number of uh all these
00:38:39.119 labs so if you go to quick cloud you go
00:38:41.440 to the catalog
00:38:43.200 and you search for vertex ai you should
00:38:46.640 see
00:38:47.599 a number of uh
00:38:50.320 labs that you can basically build and
00:38:52.400 deploy
00:38:53.440 uh machine learning solutions and vertex
00:38:55.359 ai so that is
00:38:56.800 this quest for example is an easy way to
00:38:59.440 basically go through and learn
00:39:01.920 all of the steps and i know it says 35
00:39:04.880 credits but
00:39:06.320 quick labs usually runs all of these
00:39:08.079 promotions where they basically give you
00:39:10.320 like a month free just watch for those
00:39:12.960 and that that that's that's another way
00:39:15.599 to basically learn this thing for free
00:39:21.680 awesome great
00:39:23.119 uh
00:39:25.119 again uh just remind uh you for
00:39:27.280 questions feel free to post in the chat
00:39:30.960 i see a lot of messages in the chat so
00:39:32.880 if your questions got
00:39:34.800 screwed up
00:39:36.320 [Music]
00:39:37.440 lauren has an interesting question you
00:39:38.880 can
00:39:43.920 ai are containers instances under the
00:39:46.000 hood of the endpoint uniform uh so you
00:39:49.680 get to choose so whenever you basically
00:39:51.920 deploy to an endpoint you basically get
00:39:55.040 to choose how you're basically deploying
00:39:57.760 so when you notice uh in my trained at
00:40:00.960 vertex right uh i basically specify the
00:40:04.720 machine type and i specify the scaling
00:40:07.920 i'm in this case i'm saying don't scale
00:40:10.079 because i'm not start minimum of one max
00:40:12.800 of one but you can obviously specify
00:40:14.800 auto scaling here your machine type can
00:40:17.280 be any machine temp that's supported on
00:40:19.119 google cloud plus any accelerator type
00:40:22.160 which you saw when i did the training i
00:40:23.839 said train on nvidia t4 so
00:40:27.200 yes
00:40:28.000 regardless of what the model is you can
00:40:30.160 basically do it on all of these
00:40:45.599 awesome
00:40:46.640 all right
00:40:49.680 for inferencing what feature store do
00:40:52.800 you use it has to scale
00:40:55.040 well we do have this thing called a
00:40:56.960 vertex
00:40:58.319 ai feature store
00:41:05.280 so take a look
00:41:07.520 okay
00:41:13.839 okay
00:41:16.000 uh with automl how do you see this
00:41:18.480 disrupting jobs in the data science
00:41:20.800 machine learning industry look here's
00:41:22.720 the deal though as a data scientist what
00:41:25.200 are you spending most of your time on
00:41:27.359 it's in understanding the problem it's
00:41:30.079 in evaluating models it's in preparing
00:41:34.000 data
00:41:35.359 i mean
00:41:36.319 it's not in creating the model right
00:41:38.240 that writing that model could face that
00:41:40.240 what everybody does if you're doing
00:41:41.440 image classification you're using
00:41:43.280 resonant right
00:41:44.800 if if that gets replaced by r2ml that's
00:41:47.440 replacing a really small part of the
00:41:50.160 overall workflow and now automl doesn't
00:41:54.160 replace data scientists it basically
00:41:56.960 gives you a faster way to go to market
00:42:00.560 to benchmark your models etc
00:42:13.200 for iot it has been millions per second
00:42:15.200 yes uh absolutely so
00:42:17.200 now for iot on google cloud folks use
00:42:20.079 something called data flow uh and you
00:42:22.560 basically do those in in a in a
00:42:24.960 streaming pipeline and data flow so look
00:42:27.599 up data flow streaming pipelines
00:42:31.760 machine learning uh we've had uh for
00:42:34.960 example like we've had uh you know
00:42:37.920 companies like for example telus does
00:42:40.240 the scale that you're talking about
00:42:42.240 where they're looking at millions of
00:42:44.160 events per second they basically do it
00:42:46.640 with like apache beam and tensorflow so
00:42:49.599 so take a look
00:43:00.319 all right i
00:43:01.520 thank you very much uh i think we are uh
00:43:05.040 out on times
00:43:06.400 uh
00:43:07.440 great thanks like for the uh
00:43:09.760 presentations and the
00:43:12.079 the discussions
00:43:13.520 with attendees thanks everyone for
00:43:16.400 asking the questions too
00:43:18.410 [Music]
00:43:19.760 with
00:43:20.560 with that we
00:43:22.560 conclude these sessions
00:43:25.200 i like to have anything to add on before
00:43:27.520 we
00:43:28.240 nothing thank you very much bill for the
00:43:30.079 opportunity thanks
00:43:31.599 everyone for the great questions and for
00:43:33.760 i know online is really really hard so
00:43:36.400 thanks for sticking through it and uh
00:43:38.720 and
00:43:39.520 you know to

"""

# Prompt Enginieering

In [24]:

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=M_PbbMVg4ME&t=1s",
    chunk_size_seconds=45,
    transcript_format=TranscriptFormat.CHUNKS,
    add_video_info=True,

)

transcript_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=4000,
    chunk_overlap=1000,
    length_function=len,
    is_separator_regex=False,
)

In [25]:
transcript_documents[1].metadata

{'source': 'https://www.youtube.com/watch?v=neBZ6huolkg&t=45s',
 'title': 'Build an AI/ML Football Analysis system with YOLO, OpenCV, and Python',
 'description': 'Unknown',
 'view_count': 369713,
 'thumbnail_url': 'https://i.ytimg.com/vi/neBZ6huolkg/hq720.jpg',
 'publish_date': '2024-04-18 00:00:00',
 'length': 16116,
 'author': 'Code In a Jiffy',
 'start_seconds': 45,
 'start_timestamp': '00:00:45'}

In [11]:
from io import StringIO

# Assuming your chunk objects are stored in a list called `chunks`

# Create a StringIO object to hold the content in memory
memory_file = StringIO()

# Define the output file name
output_file = 'youtube_script.txt'
# Open the file in write mode

chunks = transcript_documents
with open(output_file, 'w') as file:
    # Iterate over each chunk in the list with an index to access the next chunk
    for i in range(len(chunks)):
        # Extract metadata and content from the current chunk
        title = chunks[i].metadata.get('title', 'Unknown Title')
        start_timestamp = chunks[i].metadata.get('start_timestamp', 'Unknown Timestamp')
        page_content = chunks[i].page_content

        # Determine the end timestamp
        if i + 1 < len(chunks):
            end_timestamp = chunks[i + 1].metadata.get('start_timestamp', 'Unknown Timestamp')
        else:
            end_timestamp = "End of Video"  # or some appropriate end marker

        title = chunks[i].metadata.get('title')
        # Prepare the content to write
        content = (
            f"Video Extract : {title}\n"
            f"[{start_timestamp}---->{end_timestamp}]:\n{page_content}\n"
            f"\n"
        )

        # Write the content to both the file and the StringIO object
        file.write(content)
        memory_file.write(content)

# Get the content from the StringIO object as a string
memory_content = memory_file.getvalue()

# Print confirmation message
print(f"Script written to {output_file}")

# You can use `memory_content` as needed, it's the full script content in memory
print(memory_content)


Script written to youtube_script.txt
Video Extract : Build an AI/ML Football Analysis system with YOLO, OpenCV, and Python
[00:00:00---->00:00:45]:
hello in this video I will teach you how you can build this football analysis project from scratch this project detects and tracks players referees and footballs across the whole video using YOLO YOLO is one of the best AI object detection models right now we will also be training it to improve the output of the outof thee boox models then we'll assign players to teams according to the T-shirt colors that they are wearing this will require us to segment and cluster pixels with K means to only choose t-shirt pixels from a player's bounding box from here we can measure a team's ball acquisition percentage in a match then using optical flow we will measure how much camera movement from

Video Extract : Build an AI/ML Football Analysis system with YOLO, OpenCV, and Python
[00:00:45---->00:01:30]:
one frame to another so we can precisely measure

In [12]:
out = loader.load_and_split(text_splitter)

In [13]:
llm = ChatOpenAI(temperature=0,max_tokens=4096)


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=4000,
    chunk_overlap=1000,
    length_function=len,
    is_separator_regex=False,
)


In [14]:


chunks = text_splitter.split_text(memory_content)
documents = text_splitter.create_documents(chunks)



In [15]:

initial_chunk_prompt_template = """
Task: Here is a partial transcript of a youtube video with timestamps. Your task is to destile the information from  the raw transcript and write a concise well structure article that summarizes the
      new or intersting information. The timestamps where the information is being extracted should be added at the end. In case part or the whole of the video is a tutorial it is important to list the steps provided. 
      The reader might then go to the timestamps to check so include a chronological list or timeline summarizing the sequence of topics discussed in the video.
Part of Transcription:
{text}
PARTIAL ARTICLE:

[start_timestamp---->end_timestamp]"""


In [16]:

intial_prompt = PromptTemplate.from_template(initial_chunk_prompt_template)

refine_chunk_template = """
We are working on processing the information from a long youtube transcript and destile it into one well structure cohesive text either a Medium Article or a scientific paper whatever fits best. The idea is that a reader with a technical backgroud
can easily read our composition and understand what new information/technology/insight is being presented in the video.

We are working by parts because the transcript it's a very big.
We have made a partial document up to a certain point of the transcript: {existing_answer}

Your job is to merge and maybe if appropiate summarize this partial articles together into a consice coherent document with an abstract, introduction, body paragraphs and conclusion

Requirements for the document:
Focus on what new knowledege is the video presenting. Explain what new or intersting, technologies,tool,algorithm,discovery, are being discussed and how do they work and why are they relevant

The document should have an abstract,introduction, and a paragraph for each key topic/tool/algorithm/etc.
The body paragraphs should  reference the timestamps of the transcript that support what they are saying as if they were citations.
The reader might then go to the timestamps to check so include a chronological list or timeline summarizing the sequence of topics discussed in the video

In case the video or part of the video is a tutorial list the steps provided.


Continue and refine the existing partial article with  more context below if appropiate in a way that is still coherent. Otherwise return the current partial article
------------
{text}
------------


PARTIALLY PROCESS DOCUMENT:"""
refine_chunk_prompt = PromptTemplate.from_template(refine_chunk_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=intial_prompt,
    refine_prompt=refine_chunk_prompt,
    return_intermediate_steps=False,
    input_key="input_documents",
    output_key="output_text",
)
result = chain.invoke({"input_documents": documents}, return_only_outputs=False)



In [17]:

print(result['output_text'])


Abstract:
The video tutorial "Build an AI/ML Football Analysis system with YOLO, OpenCV, and Python" provides a comprehensive guide on creating a football analysis project using YOLO for object detection. It covers various aspects such as training YOLO, object tracking, and drawing enhancements to improve visual representation in video frames. The tutorial caters to both beginners and experienced machine learning engineers, offering insights into advanced techniques like smart tracking and trajectory prediction. This document aims to summarize and analyze the key topics discussed in the video, highlighting the new technologies and methodologies presented.

Introduction:
The tutorial focuses on building an AI/ML Football Analysis system with YOLO, OpenCV, and Python, emphasizing the importance of object detection and tracking in sports analytics. It provides step-by-step instructions on setting up the project, importing the video dataset, and utilizing YOLO for accurate object detection

In [ ]:
len(documents)

17

In [ ]:
result['intermediate_steps']

['In the youtube video "DevFest21 - AI/ML: Leveraging Prefabricated Pipelines for MLOps in Vertex AI", the speaker discusses the use of prefabricated pipelines for MLOps in Google Cloud\'s Vertex AI platform. The speaker highlights the benefits of using these pre-built pipelines for machine learning operations, such as saving time and resources by streamlining the process of deploying and managing machine learning models (00:00-02:30).\n\nThe speaker explains how prefabricated pipelines in Vertex AI can help data scientists and machine learning engineers focus on building and improving models, rather than spending time on infrastructure and deployment tasks. By leveraging these pre-built pipelines, teams can accelerate the development and deployment of machine learning models, ultimately leading to faster innovation and improved productivity (02:30-04:45).\n\nFurthermore, the speaker discusses how prefabricated pipelines in Vertex AI are customizable and can be tailored to specific use